In [1]:
# Referred https://www.pyimagesearch.com/2017/NUM_CLASSES/11/image-classification-with-keras-and-deep-learning/
# https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Activation, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import random
import os
import sys
import cv2
from keras.utils import to_categorical
import matplotlib.pyplot

# kaggle/python docker image: https://github.com/kaggle/docker-python
# Input data files are available in the "../input/" directory.
from subprocess import check_output
#list the files in the input directory
#print(check_output(["ls", "../input"]).decode("utf8"))
#print(check_output(["pwd",""]).decode("utf8")) # returns /kaggle/working
#classes = check_output(["ls", "../input/train"]).decode("utf8") # returns 12 directories
#print((classes))
def classes_to_int(label):
    # label = classes.index(dir)
    label = label.strip()
    if label == "Black-grass":  return 0
    if label == "Charlock":  return 1
    if label == "Cleavers":  return 2
    if label == "Common Chickweed":  return 3
    if label == "Common wheat":  return 4
    if label == "Fat Hen":  return 5
    if label == "Loose Silky-bent": return 6
    if label == "Maize":  return 7
    if label == "Scentless Mayweed": return 8
    if label == "Shepherds Purse": return 9
    if label == "Small-flowered Cranesbill": return 10
    if label == "Sugar beet": return 11
    print("Invalid Label", label)
    return 12

def int_to_classes(i):
    if i == 0: return "Black-grass"
    elif i == 1: return "Charlock"
    elif i == 2: return "Cleavers"
    elif i == 3: return "Common Chickweed"
    elif i == 4: return "Common wheat"
    elif i == 5: return "Fat Hen"
    elif i == 6: return "Loose Silky-bent"
    elif i == 7: return "Maize"
    elif i == 8: return "Scentless Mayweed"
    elif i == 9: return "Shepherds Purse"
    elif i == 10: return "Small-flowered Cranesbill"
    elif i == 11: return "Sugar beet"
    print("Invalid class ", i)
    return "Invalid Class"

Using TensorFlow backend.
/home/dheeraj/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/dheeraj/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/dheeraj/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/dheeraj/.local/lib/python3.6/site-packages/tensorflow/py

In [2]:
#The Plant Seedlings Dataset contains images of approximately 960 unique plants belonging to
# 12 species at several growth stages.
# It comprises annotated RGB images with a physical resolution of roughly 10 pixels per mm.
NUM_CLASSES = 12
# we need images of same size so we convert them into the size
WIDTH = 128
HEIGHT = 128
DEPTH = 3
inputShape = (WIDTH, HEIGHT, DEPTH)
# initialize number of epochs to train for, initial learning rate and batch size
EPOCHS = 15
INIT_LR = 1e-3
BS = 32

def readTrainData(trainDir):
    data = []
    labels = []
    # loop over the input images
    dirs = os.listdir(trainDir) 
    for dir in dirs:
        absDirPath = os.path.join(os.path.sep,trainDir, dir)
        images = os.listdir(absDirPath)
        for imageFileName in images:
            # load the image, pre-process it, and store it in the data list
            imageFullPath = os.path.join(trainDir, dir, imageFileName)
            #print(imageFullPath)
            img = load_img(imageFullPath)
            arr = img_to_array(img)  # Numpy array with shape (233,233,3)
            arr = cv2.resize(arr, (HEIGHT,WIDTH)) #Numpy array with shape (HEIGHT, WIDTH,3)
            #print(arr.shape) 
            data.append(arr)
            label = classes_to_int(dir)
            labels.append(label)
    return data, labels



In [3]:
def createModel():
    model = Sequential()
    # first set of CONV => RELU => POOL layers
    # The CONV  layer will learn 20 convolution filters, each of which are 5×5.
    model.add(Conv2D(20, (5, 5), padding="same", input_shape=inputShape))
    # We then apply a ReLU activation function followed by 2×2 max-pooling in both 
    # the x and y direction with a stride of two. 
    #To visualize this operation, consider a sliding window that “slides” across 
    #the activation volume, taking the max operation over each region, while taking 
    #a step of two pixels in both the horizontal and vertical direction.
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # second set of CONV => RELU => POOL layers
    #This time we are learning 50 convolutional filters rather than the 20 convolutional
    #filters as in the previous layer set. It’s common to see the number of CONV 
    #filters learned increase the deeper we go in the network architecture.
    model.add(Conv2D(50, (5, 5), padding="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # first (and only) set of FC => RELU layers
    # Flattening out the volume into a set of fully-connected layers
    # Take the output of the preceding MaxPooling2D layer and flatten it into a single vector.
    # This operation allows us to apply our dense/fully-connected layers.
    # Fully-connected layer contains 500 nodes which is passed through another 
    # nonlinear ReLU activation.
    model.add(Flatten())
    model.add(Dense(500))
    model.add(Activation("relu"))
    # softmax classifier
    # Another fully-connected layer, but this one is special — the number of nodes is equal 
    # to the number of classes  (i.e., the classes we want to recognize).
    # This Dense layer is then fed into our softmax classifier
    # which will yield the probability for each class.
    model.add(Dense(output_dim=12))
    model.add(Activation("softmax"))
    # returns our fully constructed deep learning + Keras image classifier 
    opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
    # use binary_crossentropy if there are two classes
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
    return model



In [4]:
random.seed(10)
allLabels =  os.listdir("/home/dheeraj/Documents/train/")  # list of subdirectories and files
print("Loading images...")
sys.stdout.flush()
X, Y = readTrainData("/home/dheeraj/Documents/train/")
# scale the raw pixel intensities to the range [0, 1]
X = np.array(X, dtype="float") / 255.0
Y = np.array(Y)
# convert the labels from integers to vectors
Y =  to_categorical(Y, num_classes=12)

print("Partition data into 75:25...")
sys.stdout.flush()
# partition the data into training and testing splits using 75% training and 25% for validation
(trainX, valX, trainY, valY) = train_test_split(X,Y,test_size=0.25, random_state=10)

#construct the image generator for data augmentation
print("Generating images...")
sys.stdout.flush()
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, \
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,\
    horizontal_flip=True, fill_mode="nearest")

# initialize the model
print("compiling model...")
sys.stdout.flush()
model = createModel()
# train the network
print("training network...")
sys.stdout.flush()
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS), \
    validation_data=(valX, valY), \
    steps_per_epoch=len(trainX) // BS, epochs=EPOCHS, verbose=1)

# save the model to disk
print("Saving model to disk")
sys.stdout.flush()
model.save("/tmp/mymodel")

print('Model successfully trained')


Loading images...
Partition data into 75:25...
Generating images...
compiling model...

training network...


/home/dheeraj/.local/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=12)`



Epoch 1/15
111/111 [==============================] - 53s 473ms/step - loss: 2.4928 - accuracy: 0.1436 - val_loss: 2.3959 - val_accuracy: 0.2189
Epoch 2/15
111/111 [==============================] - 52s 471ms/step - loss: 2.1691 - accuracy: 0.2606 - val_loss: 2.1196 - val_accuracy: 0.2449
Epoch 3/15
111/111 [==============================] - 52s 471ms/step - loss: 1.5626 - accuracy: 0.4448 - val_loss: 1.3364 - val_accuracy: 0.5556
Epoch 4/15
111/111 [==============================] - 52s 470ms/step - loss: 1.3986 - accuracy: 0.5105 - val_loss: 1.1252 - val_accuracy: 0.5985
Epoch 5/15
111/111 [==============================] - 53s 473ms/step - loss: 1.3156 - accuracy: 0.5462 - val_loss: 1.1664 - val_accuracy: 0.6052
Epoch 6/15
111/111 [==============================] - 52s 470ms/step - loss: 1.2318 - accuracy: 0.5592 - val_loss: 1.0661 - val_accuracy: 0.6456
Epoch 7/15
111/111 [==============================] - 52s 468ms/step - loss: 1.2133 - accuracy: 0.5824 - val_loss: 1.0211 - val_a

In [17]:
def readTestData(testDir):
    data = []
    filenames = []
    # loop over the input images
    images = os.listdir(testDir)
    for imageFileName in images:
        # load the image, pre-process it, and store it in the data list
        imageFullPath = os.path.join(testDir, imageFileName)
        #print(imageFullPath)
        img = load_img(imageFullPath)
        arr = img_to_array(img)  # Numpy array with shape (...,..,3)
        arr = cv2.resize(arr, (HEIGHT,WIDTH)) 
        data.append(arr)
        filenames.append(imageFileName)
    return data, filenames

# read test data and find its classification
import tkinter
from tkinter import filedialog
import os

root = tkinter.Tk()
root.withdraw() #use to hide tkinter window

def search_for_file_path ():
    currdir = os.getcwd()
    tempdir = filedialog.askdirectory(parent=root, initialdir=currdir, title='Please select a directory')
    if len(tempdir) > 0:
        print ("You chose: %s" % tempdir)
    return tempdir


file_path_variable = search_for_file_path()
print ("\nfile_path_variable = ", file_path_variable)


testX, filenames = readTestData(str(file_path_variable))
# scale the raw pixel intensities to the range [0, 1]
testX = np.array(testX, dtype="float") / 255.0

from keras.models import load_model
mymodel = load_model('/tmp/mymodel')
yFit = mymodel.predict(testX, batch_size=10, verbose=1)

#print(type(yFit)) # numpy.ndarray
#print(type(filenames)) # list

import csv  
with open('output.csv', 'w', newline='') as csvfile:
    fieldnames = ['file', 'species']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for index, file in enumerate(filenames):
        classesProbs = yFit[index]
        maxIdx = 0
        maxProb = 0;
        for idx in range(0,11):
            if(classesProbs[idx] > maxProb):
                maxIdx = idx
                maxProb = classesProbs[idx]
        writer.writerow({'file': file, 'species': int_to_classes(maxIdx)})
print("Writing complete")


You chose: /home/dheeraj/Documents/anothertest

file_path_variable =  /home/dheeraj/Documents/anothertest
3/3 [==============================] - 0s 116ms/step
Writing complete


In [17]:
from tkinter import *
import pandas 
root=tk()
df=pandas.read_csv('output.csv')
w=Label(root, text=str(df))
w.pack()
root.mainloop()

                         file           species
0                  Sugar Beet  Loose Silky-bent
1             Shepherds Purse  Common Chickweed
2                Common Wheat  Loose Silky-bent
3   Small flowered Cranesbill  Common Chickweed
4                       Maize  Common Chickweed
5           Scentless Mayweed  Common Chickweed
6            Loose Silky Bent  Common Chickweed
7                    Cleavers  Loose Silky-bent
8                     Fat Hen  Loose Silky-bent
9                    Charlock  Common Chickweed
10           Common Chickweed  Common Chickweed
11                Black grass  Loose Silky-bent
